In [1]:
import transformers
import torch
from transformers import AutoTokenizer, LlamaTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast
# from models.modeling_llama import LlamaForCausalLM
# from models.yarn_llama.modeling_llama_together_yarn import LlamaForCausalLM as yarnLlamaForCausalLM
# from models.chatglm.modeling_chatglm import ChatGPT3LMHeadModel
# from models.mistral.modeling_mistral import MistralForCausalLM
import torch.nn as nn
model = AutoModelForCausalLM.from_pretrained("/dev/shm/Meta-Llama-3-8B/")
# print(model)
# for name, module in model.named_children():
#     print(name)
#     current_key_name = []
#     current_key_name.append(name)
#     if (isinstance(module, nn.Linear)):
#         in_features = module.in_features
#         out_features = module.out_features
#         weight = module.weight
#         bias = module.bias
#         print(in_features)
#         print(out_features)
#         print(weight.shape)
#         print(bias)
for layer in model.model.layers:
    print(layer.self_attn)
    print(model.in_features)
    in_features = module.in_features
    out_features = module.out_features
    weight = module.weight
    bias = module.bias
        
    #     model._modules[name] = QLinear(
    #         in_features,
    #         out_features,
    #         module.bias is not None,
    #         compute_dtype=compute_dtype,
    #         quant_type=quant_type,
    #         q_group_size=q_group_size
    #     )

    #     model._modules[name].weight = weight
    #     model._modules[name].bias = bias
    #     has_been_replaced = True
    #     # Store the module class in case we need to transpose the weight later
    #     model._modules[name].source_cls = type(module)
    # if len(list(module.children())) > 0:
    #     _, has_been_replaced = convertModelToQuant(
    #         module,
    #         modules_to_not_convert,
    #         current_key_name,
    #         has_been_replaced,
    #         compute_dtype,
    #         quant_type,
    #         q_group_size
    #     )
    # # Remove the last key for recursion
    # current_key_name.pop(-1)

/home/superbench/anaconda3/envs/zhichenz/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.05it/s]

model
lm_head
4096
128256
torch.Size([128256, 4096])
None


In [96]:
import transformers
import torch
from transformers import AutoTokenizer, LlamaTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast
import torch.nn as nn
from torch.autograd import Function

class Round(Function):
    @staticmethod
    def forward(self, input):
        sign = torch.sign(input)
        output = sign * torch.floor(torch.abs(input) + 0.5)
        return output

    @staticmethod
    def backward(self, grad_output):
        grad_input = grad_output.clone()
        return grad_input
    
class SteInt2AsymQuantizer(nn.Module):
    def __init__(self, q_group_size=128):
        super().__init__()
        self.q_group_size = q_group_size
        self.bit = 16
    def forward(self, x):
        org_w_shape = x.shape

        if self.q_group_size > 0:
            assert org_w_shape[-1] % self.q_group_size == 0
            x = x.reshape(-1, self.q_group_size)
            print(x.shape)
        assert x.dim() == 2

        max_val = x.amax(dim=1, keepdim=True)
        min_val = x.amin(dim=1, keepdim=True)
        max_int = 2 ** self.bit - 1
        min_int = 0
        scales = (max_val - min_val).clamp(min=1e-5) / max_int
        zeros = (-torch.round(min_val / scales)).clamp_(min_int, max_int)

        assert torch.isnan(scales).sum() == 0
        assert torch.isnan(x).sum() == 0

        x = (torch.clamp(Round.apply(x / scales) +
                         zeros, min_int, max_int) - zeros) * scales
        assert torch.isnan(x).sum() == 0

        x = x.reshape(org_w_shape)

        return x
bs, seq, num_heads, head_dim = 1, 2, 8, 128
tensor = torch.randn(bs, seq, num_heads, head_dim)
new_tensor = tensor.clone()
quant = SteInt2AsymQuantizer()
new_tensor=quant(new_tensor)
# compare with the original
# print((tensor-new_tensor)/tensor)
similarity = torch.nn.functional.cosine_similarity(tensor.flatten(), new_tensor.flatten(), dim=0)
print(similarity)



torch.Size([16, 128])
tensor(1.0000)


In [8]:
import deepspeed
deepspeed.ops.op_builder.CPUAdamBuilder().load()

Installed CUDA version 12.2 does not match the version torch was compiled with 12.1 but since the APIs are compatible, accepting this combination


Using /home/superbench/.cache/torch_extensions/py39_cu121 as PyTorch extensions root...
No modifications detected for re-loaded extension module cpu_adam, skipping build step...
Loading extension module cpu_adam...


ImportError: /home/superbench/.cache/torch_extensions/py39_cu121/cpu_adam/cpu_adam.so: cannot open shared object file: No such file or directory